In [1]:
%reload_ext lab_black
import pandas as pd
import numpy as np
from plotnine import *

In [2]:
# read data
od = pd.read_csv(
    "https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/master/20_intermediate_files/vitalitydf_allcounties.csv?token=ARFW6V3M5NHDYLJJWTP7TEK7WMMEO"
)

# no longer need to merge in population since it was done in the code that cleaned the Vital Stats data, but keeping it in here in case needed (#remnant)
# population = pd.read_csv("https://raw.githubusercontent.com/MIDS-at-Duke/estimating-impact-of-opioids-2020-purlple-team/ssloate/20_intermediate_files/countypopulations_clean.csv?token=ARFW6V42CGGQPTDC3HUFNAS7WMLUO")

In [20]:
# remnant

# rename od columns to match
# od = od.rename(columns={"County Code": "FIPS"})
# od = od.rename(columns={"County Code": "FIPS"})

# drop _merge column in popualtion dataset
# population = population.drop(columns=["_merge", "Unnamed: 0"])

In [21]:
# remove Alaska from od Data
od = od[od["State"] != "AK"]

In [22]:
# remnant

# merge dataset
# od_pop = pd.merge(population, od, on=["FIPS", "Year"], how="outer", indicator=True)

# assert od_pop[od_pop["_merge"] == "both"].all

In [23]:
od_pop = od

In [24]:
# convert years to integers
od_pop["Year"] = od_pop["Year"].astype("int")
od_pop.sample(25)

,FIPS,Year,Deaths,County,Population,State,death rate,deathcoef,projdeaths,CalculatedDeaths
14489,22055,2010,30.0,Lafayette Parish,222315,LA,0.000122,27.0,9.0,30.0
39441,55127,2015,13.0,Walworth County,102475,WI,0.000162,17.0,9.0,13.0
32817,48161,2008,NaN,Freestone County,19446,TX,0.000081,2.0,2.0,2.0
28889,42115,2006,NaN,Susquehanna County,43376,PA,0.000142,6.0,6.0,6.0
34573,48431,2009,NaN,Sterling County,1160,TX,0.000095,0.0,0.0,0.0
2653,6095,2004,18.0,Solano County,409301,CA,0.000097,40.0,9.0,18.0
6837,16001,2015,44.0,Ada County,432862,ID,0.000130,56.0,9.0,44.0
25847,39049,2006,179.0,Franklin County,1115924,OH,0.000142,158.0,9.0,179.0
24177,36115,2013,NaN,Washington County,62765,NY,0.000120,8.0,8.0,8.0
35626,50027,2009,NaN,Windsor County,56708,VT,0.000122,7.0,7.0,7.0


In [25]:
# drop unneeded columns

od_pop = od_pop.drop(columns=["Deaths"])

In [28]:
# #remnant- rename columns for easier understanding
# od_pop = od_pop.rename(columns={"State Name": "State", "County_x": "County"},errors="raise",)

# reorder columns
od_pop = od_pop[["FIPS", "State", "County", "Year", "Population", "CalculatedDeaths"]]

# sort by state, county
od_pop = od_pop.sort_values(["State", "County", "Year"])
od_pop.sample(25)

,FIPS,State,County,Year,Population,CalculatedDeaths
17655,27141,MN,Sherburne County,2004,78091,4.0
7057,16035,ID,Clearwater County,2014,8561,1.0
37901,54003,WV,Berkeley County,2009,102830,15.0
6085,13219,GA,Oconee County,2004,27904,2.0
34545,48427,TX,Starr County,2007,59529,5.0
16515,26131,MI,Ontonagon County,2008,6922,1.0
18318,28069,MS,Kemper County,2004,10648,2.0
7241,16065,ID,Madison County,2003,29997,2.0
17568,27127,MN,Redwood County,2008,16103,1.0
26695,40003,OK,Alfalfa County,2009,5585,1.0


In [29]:
# check for null values
od_pop[od_pop["State"].isnull()]

,FIPS,State,County,Year,Population,CalculatedDeaths


In [125]:
# FIPS 51515 and 51560 represent two counties in Virginia, Bedford City and Clifton Forge Valley, that were incorporated in to other counties. It is safe to drop them

od_pop = od_pop[(od_pop["FIPS"] != 51515) & (od_pop["FIPS"] != 51560)]
od_pop[od_pop["State"].isnull()]

,FIPS,State Abbr,State,County,Year,Deaths,Population


In [126]:
# Create per capita deaths column
od_pop["Deaths Per 100,000 People"] = (od_pop["Deaths"] / od_pop["Population"]) * 100000

In [128]:
# save to excel

od_pop.to_csv(
    "/Users/samsloate/Desktop/Data_Science/Opioids_Project/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/death_data_with_pop.csv"
)